In [ ]:
########################################################################
#    July 03, 2019                                                     #
#    11:45                                                             #
#    Created by: Ankit Chillar, Kunal Gehlot                           #
#    Company: Panda Projects                                           #
########################################################################

### SET PATH FOR THE DIRECTORY FOLDER

In [1]:
SAMPLE_PATH = 'D:\OneDrive\Documents\PANDA\siim-acr-pneumothorax-segmentation\sample images\*.dcm'
DATASET_PATH = 'D:\OneDrive\Documents\PANDA\siim-acr-pneumothorax-segmentation\dicom-images-train\*\*\*.dcm'
TEST_PATH = 'D:\OneDrive\Documents\PANDA\siim-acr-pneumothorax-segmentation\dicom-images-test\*\*\*.dcm'
RLE_SAMPLE = 'D:/OneDrive/Documents/PANDA/siim-acr-pneumothorax-segmentation/sample images/train-rle-sample.csv'
RLE_PATH = 'D:/OneDrive/Documents/PANDA/siim-acr-pneumothorax-segmentation/train-rle.csv'

### IMPORT LIBRARIES 

In [2]:
import os
import pydicom
import pandas as pd
import numpy as np
import tensorflow as tf
import glob
import cv2
import sys
import keras.layers as kl
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from collections import defaultdict
from pydicom.data import get_testdata_files
from glob import glob
from tqdm import tqdm
from IPython.display import Audio, display
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, concatenate, MaxPooling2D, Conv2DTranspose, Activation
from collections import defaultdict
from pydicom.data import get_testdata_files
from keras.models import Sequential
from sklearn.model_selection import train_test_split

gpu_options = tf.GPUOptions(allow_growth=True)
session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))
tf.Session()
tf.device('/gpu:0')


def allDone():
  display(Audio(url='Siren.wav', autoplay=True))


Using TensorFlow backend.


### Extracting Pixels 

In [3]:
def PixelArray(dataset, figsize=(10,10)):
    plt.figure(figsize=figsize)
    plt.imshow(dataset.pixel_array, cmap=plt.cm.bone)
    print(dataset.pixel_array)
    print(plt.cm.bone)
    plt.show()

### MASK2RLE AND RLE2MASK FUNCTIONS

In [4]:
def mask2rle(img, width, height):
    rle = []
    lastColor = 0;
    currentPixel = 0;
    runStart = -1;
    runLength = 0;

    for x in range(width):
        for y in range(height):
            currentColor = img[x][y]
            if currentColor != lastColor:
                if currentColor == 255:
                    runStart = currentPixel;
                    runLength = 1;
                else:
                    rle.append(str(runStart));
                    rle.append(str(runLength));
                    runStart = -1;
                    runLength = 0;
                    currentPixel = 0;
            elif runStart > -1:
                runLength += 1
            lastColor = currentColor;
            currentPixel+=1;

    return " ".join(rle)

def rle2mask(rle, width, height):
    mask= np.zeros(width* height)
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]
    lengths = array[1::2]

    current_position = 0
    for index, start in enumerate(starts):
        current_position += start
        mask[current_position:current_position+lengths[index]] = 255
        current_position += lengths[index]

    return mask.reshape(width, height)

# SORTING THE TRAIN DATASET

In [5]:
# filename = get_testdata_files('rtplan.dcm')[0]
files = sorted(glob(DATASET_PATH))

# CHECKING THE NO OF FILES IMPORTED

In [6]:
len(files)

10712

# READ THE TRAIN RLE FILE

In [7]:
RLEs = pd.read_csv(RLE_PATH)

# PRINTING THE LENGTH OF THE RLE FILE

In [8]:
len(RLEs)

11582

# CREATING A LIST TO STORE THE RLE FOR DATA CLEANING

In [9]:
RLEsL = defaultdict(list)

# CHECKING THE NO OF SAMLES WHICH ARE ANNOATED

In [10]:
for image_id, rle in zip(RLEs['ImageId'], RLEs[' EncodedPixels']):
    RLEsL[image_id].append(rle)
annotated = {k: v for k, v in RLEsL.items() if v[0] != ' -1'}
print("%d of %d images are annotated (Positive) " % (len(annotated), len(RLEsL)))

2379 of 10675 images are annotated (Positive) 


# VALUES WITH NO VALUES

In [11]:
print('Missing values are: (DROP THESE)', len(files) - len(RLEsL))

Missing values are: (DROP THESE) 37


# DATA CLEANING BY REMOVING THE ABOVE FILES

In [12]:
count = 0
for i in files:
    
#     print(RLEs.ImageId.str.contains(i.split('\\')[-1][:-4]))
    if RLEs.ImageId.str.contains(i.split('\\')[-1][:-4]).any():
        count = count
    else:
        files.remove(i)
        #print("False")
        count += 1
        

# FOR DROPPING THE DUPLICATE FILES FROM THE DATASET

In [13]:
indices = RLEs[RLEs.duplicated(subset='ImageId', keep='first')]

# STORING THE RLES AND INDICES IN ds1 AND ds2 RESPECTIVELY

In [14]:
ds1 = set([tuple(line) for line in RLEs.values])
ds2 = set([tuple(line) for line in indices.values])

# PRINTING THE LENGTH OF THE DATASET AFTER REMOVING FILES

In [15]:
len(ds1.difference(ds2))

10675

In [16]:
RLEs = pd.DataFrame(list(ds1.difference(ds2)), columns = ['ImageId', ' EncodedPixels'])

In [17]:
RLEs = RLEs.set_index('ImageId')

# FOR VISUALIZING A PART OF THE TRAIN DATASET

In [ ]:
print("Visualizing the part of the full dataset .........")

start = 20
numberOfImages = 50

fig, ax = plt.subplots(nrows = numberOfImages, ncols = 1, sharey=True, figsize=(250, 250))
for q, path in enumerate(glob(DATASET_PATH)[start:start+numberOfImages]):
    dataset = pydicom.dcmread(path)
    ax[q].imshow(dataset.pixel_array, cmap = plt.cm.bone)
    compare = RLEs[' EncodedPixels'].loc[path.split('\\')[-1][:-4]]
    print(compare != ' -1')
    if compare != ' -1':
        mask = rle2mask(RLEs[' EncodedPixels'].loc[path.split('\\')[-1][:-4]], 1024, 1024).T
        ax[q].set_title('See Marker')
        ax[q].imshow(mask, alpha=0.6, cmap='Reds')
    else:
        ax[q].set_title('Nothing to see')

# SET THE NO. OF FILES TO BE IMPORTED

In [18]:
files = files[:4000]

# FOR IMPORTING THE TRAIN IMAGES AND MASK FROM THE DATASET 

# AND CREATING THE TRAINING DATA X_train AND Y_train

In [19]:
height = 1024
width = 1024
binSeg = 1

X_train = np.zeros((len(files), height, width, binSeg), dtype=np.uint8)
Y_train = np.zeros((len(files), height, width, 1), dtype=np.bool)
#X_feat = np.zeros((len(files), 1), dtype=np.float32)

print('Getting train images and masks ... ')
sys.stdout.flush()

for n, _id in tqdm_notebook(enumerate(files), total=len(files)):
    dataset = pydicom.read_file(_id)
    
    X_train[n] = np.expand_dims(dataset.pixel_array, axis=2)
    if '-1' in RLEs.loc[_id.split('\\')[-1][:-4],' EncodedPixels']:
        Y_train[n] = np.zeros((1024, 1024, 1))
    else:
        Y_train[n] = np.expand_dims(rle2mask(RLEs.loc[_id.split('\\')[-1][:-4],' EncodedPixels'], 1024, 1024), axis=2)
print("Got'em")

Getting train images and masks ... 



Got'em


# FOR RESHAPING THE IMAGES TO FEED INTO THE U-NET MODEL

# INCREASE HEIGHT AND WIDTH TO DECREASE THE TRAIN SAMPLES

# DECREASE HEIGHT AND WIDTH TO INCREASE THE TRAIN SAMPLES

# MORE TRAINING SAMPLES MEANS LESS LOSS

In [ ]:
height = 64
width = 64
X_train = X_train.reshape((-1, height, width, 1))
Y_train = Y_train.reshape((-1, height, width, 1))

# DICE COEF FUNCTION FOR PIXEL COMPARSION

# INCREASE SMOOTH FOR GETTING THE DICE COEF CLOSE TO 1

# DECREASE SMOOTH FOR GETTING THE DICE COEF CLOSE TO 0

In [ ]:
def dice_coef(y_true, y_pred, smooth=1):
    print(binSeg, 'in dice')
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# MORE CONVOLUTIONAL LAYERS (CL) CAN BE ADDED TO MODEL TO INCREASE THE TOTAL NO OF PARAMETERS

# MORE NO OF PARAMETERS DECREASES THE LOSS 

# U-NET MODEL (CNN)

In [ ]:
inputs = Input((height, width, binSeg))

c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (inputs)
c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (p1)
c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (p2)
c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (p3)
c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (c4)
p4 = MaxPooling2D((2, 2)) (c4)

#c41 = Conv2D(64, (3, 3), activation='relu', padding='same') (p3)
#c41 = Conv2D(64, (3, 3), activation='relu', padding='same') (c41)
#p41 = MaxPooling2D((2, 2)) (c41)

c5 = Conv2D(128, (3, 3), activation='relu', padding='same') (p4)
c5 = Conv2D(128, (3, 3), activation='relu', padding='same') (c5)

#u61 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c5)
#u61 = concatenate([u61, c41])
#c61 = Conv2D(64, (3, 3), activation='relu', padding='same') (u61)
#c61 = Conv2D(64, (3, 3), activation='relu', padding='same') (c61)

u6 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(64, (3, 3), activation='relu', padding='same') (u6)
#c6 = Conv2D(64, (3, 3), activation='relu', padding='same') (c6)

u7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(32, (3, 3), activation='relu', padding='same') (u7)
#c7 = Conv2D(32, (3, 3), activation='relu', padding='same') (c7)

u8 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(16, (3, 3), activation='relu', padding='same') (u8)
#c8 = Conv2D(16, (3, 3), activation='relu', padding='same') (c8)

u9 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (u9)
#c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=[dice_coef])
model.summary()

# CALLING model.fit FOR CHECKING THE VALUE OF LOSS AND DICE COEF

# FOR MAKING THE model.h5 FOR FUTURE TRAINING

# DECREASE THE batch_size TO IMPROVING THE ACCURACY OF THE MODEL

# INCREASE THE batch_size TO REDUCING THE PROCESSING TIME

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(patience=3, verbose=1),
    tf.keras.callbacks.ModelCheckpoint('model.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

results = model.fit(X_train, Y_train, validation_split=.2, callbacks=callbacks, batch_size=25, epochs=10)

In [ ]:
#model.fit(X_train, Y_train, validation_split=.2, batch_size=50, epochs=5)

# TESTING

# LOADING THE TEST DATASET

In [ ]:
test_files = sorted(glob(TEST_PATH))

# CHECKING THE LENGTH OF THE TEST DATASET

In [ ]:
len(test_files)

# RESIZE IMAGES TO MAKE SURE ALL IMAGES ARE OF SAME SIZE

In [ ]:
height = 1024
width = 1024
binSeg = 1

X_test = np.zeros((len(test_files), height, width, binSeg), dtype=np.uint8)

for n, _id in tqdm_notebook(enumerate(test_files), total=len(test_files)):
    dataset = pydicom.read_file(_id)
    
    X_test[n] = np.expand_dims(dataset.pixel_array, axis=2)
    
X_test = X_test.reshape((-1, 64, 64, 1))

In [ ]:
Y_pred = model.predict(X_test)

# FOR VISUALIZING THE PART OF THE TEST DATA SET

In [ ]:
print("Visualizing the part of the test dataset .........")

start = 20
numberOfImages = 50

fig, ax = plt.subplots(nrows = numberOfImages, ncols = 1, sharey=True, figsize=(250, 250))
for q, path in enumerate(glob(TEST_PATH)[start:start+numberOfImages]):
    testdata = pydicom.dcmread(path)
    ax[q].imshow(testdata.pixel_array, cmap = plt.cm.bone)
    ax[q].set_title('Images')
    #print(path)
    #mask = rle2mask(RLEs[' EncodedPixels'].loc[path.split('\\')[-1][:-4]], 1024, 1024).T
    #ax[q].imshow(mask, alpha=0.6, cmap='Reds')

# LOAD THE MODEL FOR PREDICTION

In [ ]:
#model.load_weights('model.h5')

# Evaluate on validation set (this must be equals to the best log_loss)

In [ ]:
#model.evaluate({'img': X_valid, 'feat': X_feat_valid}, y_valid, verbose=1)

# Predict on train, val and test

In [ ]:
#preds_train = model.predict({'img': X_train, 'feat': X_feat_train}, verbose=1)
#preds_val = model.predict({'img': X_valid, 'feat': X_feat_valid}, verbose=1)
#preds_test = model.predict({'img': X_test, 'feat': X_feat_test}, verbose=1)

In [ ]:
allDone()